In [34]:
pip install datasets

                                              0.0/1.6 MB ? eta -:--:--
     ----------                               0.4/1.6 MB 8.9 MB/s eta 0:00:01
     --------------------------------         1.3/1.6 MB 13.8 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 14.5 MB/s eta 0:00:00
                                              0.0/262.3 kB ? eta -:--:--
     ---------------------------------------- 262.3/262.3 kB ? eta 0:00:00
  Using cached mako-1.3.10-py3-none-any.whl (78 kB)
                                              0.0/226.5 kB ? eta -:--:--
     ------------------------------------- 226.5/226.5 kB 13.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/515.2 kB ? eta -:--:--
     ------------------------------------  512.0/515.2 kB 16.2 MB/s eta 0:00:01
     ------------------------------------- 515.2/515.2 kB 10.7 MB/s eta 0:00:00
                                              0.0/27.5 MB ? eta -:--:--
     -                                        1.3/27.5 MB 42.6 MB/s eta 0:00:01
     ---                                      2.6/27.5 MB 33.3 MB/s eta 0:00:01
     -----                                    3.8/27.5 MB 30.3 MB/s eta 0:00:01
     -------                                  4.9/27.5 MB 28.6 MB/s eta 0:00:01
     ---------                                6.3/27.5 MB 28.8 MB/s eta 0:00:01
     -----------                              8.0/27.5 MB 30.3 MB/s eta 0:00:01
     ------------                             8.9/27.5 MB 28.6 MB/s eta 0:00:01
     --------------                          10.5/27.5 MB 28.5 MB/s eta 0:00:01
     -----------------                       12.5/27.5 


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [149]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import Adam
# from torch.utils.data import TensorDataset, DataLoader
# from gensim.models import Word2Vec
# from seqeval.metrics import classification_report, f1_score
# from tqdm import tqdm
# import os

from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, classification_report
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import TrainingArguments, AutoTokenizer, EvalPrediction, AutoModelForTokenClassification, Trainer
import sys
import os
import argparse
import torch
import ast
# from datasets import load_dataset, ClassLabel, Value, Sequence, load_from_disk
from tqdm import tqdm
import time
from datasets import load_dataset, ClassLabel, Value, Sequence, load_from_disk, Dataset


### `Transformer Classification`

In [29]:
SEQUENCE_LENGTH = 128
# EMBEDDING_DIM = 100
# BATCH_SIZE = 32
# EPOCHS = 100
# LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# bert_model = args.model
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 4
EPOCHS = 100
LEARNING_RATE = 0.001
bert_model = "camembert-base"
nb_labels = 2
tokenizer = AutoTokenizer.from_pretrained(bert_model)

In [70]:
W2V_PATH = "w2v_med_cbow.model"

TRAIN_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtrain_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAtrain_layer1_ID.csv",
]
VALID_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEdev_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAdev_layer1_ID.csv",
]
TEST_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtest_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAtest_layer1_ID.csv",
]

In [147]:
# --- 2. ROBUST DATA LOADER ---
def load_data_from_csv(file_paths):
    all_sents, all_tags = [], []
    for fpath in file_paths:
        if not os.path.exists(fpath):
            continue
        print(f"Loading {os.path.basename(fpath)}...", end=" ")
        try:
            df = pd.read_csv(
                fpath,
                sep=None,
                engine="python",
                keep_default_na=False,
                skip_blank_lines=False,
            )
        except:
            continue

        if "Mot" in df.columns:
            words, tags = df["Mot"].astype(str).values, df["Tag"].astype(str).values
        else:
            words, tags = (
                df.iloc[:, 0].astype(str).values,
                df.iloc[:, -1].astype(str).values,
            )

        curr_s, curr_t = [], []
        file_s, file_t = [], []

        for w, t in zip(words, tags):
            if not w.strip():
                if curr_s:
                    file_s.append(curr_s)
                    file_t.append(curr_t)
                    curr_s, curr_t = [], []
            else:
                curr_s.append(w)
                curr_t.append(t)
        if curr_s:
            file_s.append(curr_s)
            file_t.append(curr_t)

        # Chunking Fallback
        if len(file_s) < 10 and len(words) > 500:
            flat_w = [w for s in file_s for w in s]
            flat_t = [t for s in file_t for t in s]
            file_s = [flat_w[i : i + MAX_LEN] for i in range(0, len(flat_w), MAX_LEN)]
            file_t = [flat_t[i : i + MAX_LEN] for i in range(0, len(flat_t), MAX_LEN)]

        print(f"-> {len(file_s)} sentences.")
        all_sents.extend(file_s)
        all_tags.extend(file_t)
    return all_sents, all_tags


print("--- LOADING DATA ---")
train_sents, train_tags = load_data_from_csv(TRAIN_FILES)
valid_sents, valid_tags = load_data_from_csv(VALID_FILES)
test_sents, test_tags = load_data_from_csv(TEST_FILES)

--- LOADING DATA ---
Loading MEDLINEtrain_layer1_ID.csv... -> 116 sentences.
Loading EMEAtrain_layer1_ID.csv... -> 154 sentences.
Loading MEDLINEdev_layer1_ID.csv... -> 115 sentences.
Loading EMEAdev_layer1_ID.csv... -> 136 sentences.
Loading MEDLINEtest_layer1_ID.csv... -> 120 sentences.
Loading EMEAtest_layer1_ID.csv... -> 124 sentences.


In [150]:
# --- 3. TOKENIZATION ---
unique_tags = sorted(list(set(t for s in train_tags + test_tags for t in s)))
label2id = {tag: i for i, tag in enumerate(unique_tags)}
id2label = {i: tag for i, tag in enumerate(unique_tags)}

train_ds = Dataset.from_dict({"tokens": train_sents, "ner_tags": train_tags})
valid_ds = Dataset.from_dict({"tokens": valid_sents, "ner_tags": valid_tags})
test_ds = Dataset.from_dict({"tokens": test_sents, "ner_tags": test_tags})

# Try loading Fast Tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
except:
    MODEL_NAME = "almanach/camembert-base"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=MAX_LEN,
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


print("Tokenizing...")
tokenized_train = train_ds.map(tokenize_and_align_labels, batched=True)
tokenized_valid = valid_ds.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_ds.map(tokenize_and_align_labels, batched=True)

c:\Users\Tyler Marino\OneDrive\Desktop\GradSchool\Saclay\AI-Courses\TextMining-ChatBots\Venv_TextChat\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tyler Marino\.cache\huggingface\hub\models--almanach--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Tokenizing...


Map: 100%|██████████| 244/244 [00:00<00:00, 2848.31 examples/s]


In [ ]:

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    precision_weighted_average = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall_average = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')
    f1_weighted_average = 2.0 * ((precision_weighted_average * recall_average) / ( precision_weighted_average + recall_average ))
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    print(classification_report(y_pred, y_true))
    # return as dictionary
    metrics = {'precision weighted': precision_weighted_average,
               'recall weighted': recall_average,
               'f1 weighted': f1_weighted_average,
               'accuracy': accuracy}
    return metrics

def multi_label_metrics_v2(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    return (classification_report(y_true, y_pred))

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result

def padding(examples):
    for key in ['input_ids', 'attention_mask']:
        for i, element in enumerate(examples[key]):
            if len(element) < 20:
                for i in range(20 - len(element)):
                    element.append(0)
            else:
                examples[key][i] = element[:20]
    return examples


In [154]:
# --- 5. TOKENIZATION & ALIGNMENT ---
# BERT breaks words into pieces ("Hepatitis" -> "Hepa", "##titis").
# We must align tags so "Hepa" gets the label and "##titis" is ignored (-100).
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=MAX_LEN,
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens (CLS, SEP)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])  # Label the first sub-token
            else:
                label_ids.append(-100)  # Ignore subsequent sub-tokens
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)   # ignored in loss
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(-100)   # subword continuation

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


print("Tokenizing...")
tokenized_train = train_ds.map(tokenize_and_align_labels, batched=True)
tokenized_valid = valid_ds.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_ds.map(tokenize_and_align_labels, batched=True)

Tokenizing...


Map: 100%|██████████| 244/244 [00:00<00:00, 2248.09 examples/s]


In [155]:
model = AutoModelForTokenClassification.from_pretrained(bert_model, num_labels=nb_labels)
for name, param in model.named_parameters():
    print(name, param.requires_grad)

training_args = TrainingArguments(
    f"finetune_" + bert_model.replace("/","_") + "_" + str(EPOCHS) + "_" + str(TRAIN_BATCH_SIZE),
    save_strategy = "epoch",
    do_eval=True,
    eval_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=TRAIN_BATCH_SIZE,
    num_train_epochs=EPOCHS,
    load_best_model_at_end=True)


Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


roberta.embeddings.word_embeddings.weight True
roberta.embeddings.position_embeddings.weight True
roberta.embeddings.token_type_embeddings.weight True
roberta.embeddings.LayerNorm.weight True
roberta.embeddings.LayerNorm.bias True
roberta.encoder.layer.0.attention.self.query.weight True
roberta.encoder.layer.0.attention.self.query.bias True
roberta.encoder.layer.0.attention.self.key.weight True
roberta.encoder.layer.0.attention.self.key.bias True
roberta.encoder.layer.0.attention.self.value.weight True
roberta.encoder.layer.0.attention.self.value.bias True
roberta.encoder.layer.0.attention.output.dense.weight True
roberta.encoder.layer.0.attention.output.dense.bias True
roberta.encoder.layer.0.attention.output.LayerNorm.weight True
roberta.encoder.layer.0.attention.output.LayerNorm.bias True
roberta.encoder.layer.0.intermediate.dense.weight True
roberta.encoder.layer.0.intermediate.dense.bias True
roberta.encoder.layer.0.output.dense.weight True
roberta.encoder.layer.0.output.dense.bia

In [156]:
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_train, eval_dataset=tokenized_valid, tokenizer=tokenizer, compute_metrics=compute_metrics)
start = time.time()
trainer.train()
end = time.time()
elapsed = end - start

print("************** RESULTS for " + bert_model.replace("/","_") + " *****************")
print("Training time: " + str(elapsed) + " ms")
results_valid = trainer.evaluate(tokenized_valid)
print(results_valid)
results_test = trainer.evaluate(tokenized_test)
print(results_test)

C:\Users\Tyler Marino\AppData\Local\Temp\ipykernel_41712\440837247.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_train, eval_dataset=tokenized_valid, tokenizer=tokenizer, compute_metrics=compute_metrics)


: 

In [16]:
# --- 7. FINAL TEST EVALUATION ---
print("\n--- TEST RESULTS ---")
model.load_state_dict(torch.load("best_cnn_ner.pt"))
model.eval()
test_true, test_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        out = model(x)
        preds = torch.argmax(out, dim=2).cpu().numpy()
        labels = y.numpy()
        for i in range(len(x)):
            p_s, t_s = [], []
            for j in range(SEQUENCE_LENGTH):
                if labels[i][j] == 0:
                    break
                p_s.append(idx2tag[preds[i][j]])
                t_s.append(idx2tag[labels[i][j]])
            test_pred.append(p_s)
            test_true.append(t_s)

print(classification_report(test_true, test_pred))


--- TEST RESULTS ---
              precision    recall  f1-score   support

        ANAT       0.38      0.24      0.30       364
        CHEM       0.45      0.19      0.27      1037
        DEVI       0.15      0.04      0.06       107
        DISO       0.35      0.24      0.29       977
        GEOG       0.50      0.05      0.09        63
        LIVB       0.75      0.57      0.65       498
        OBJC       0.14      0.04      0.06        81
        PHEN       0.00      0.00      0.00        70
        PHYS       0.48      0.21      0.29       190
        PROC       0.41      0.49      0.45       761

   micro avg       0.44      0.30      0.35      4148
   macro avg       0.36      0.21      0.24      4148
weighted avg       0.43      0.30      0.34      4148



/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
